In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.master('local[*]').appName('BooksRatings_join').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/01 08:11:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
rating_df = spark.read.options(sep=';', header=True, inferSchema=True).csv('/home/maryam/content/BooksRating-CSV/Book-Ratings.csv')

In [9]:
rating_df.show(10)

+------+----------+----+
|userid|      isbn|rate|
+------+----------+----+
|276725|034545104X|   0|
|276726|0155061224|   5|
|276727|0446520802|   0|
|276729|052165615X|   3|
|276729|0521795028|   6|
|276733|2080674722|   0|
|276736|3257224281|   8|
|276737|0600570967|   6|
|276744|038550120X|   7|
|276745| 342310538|  10|
+------+----------+----+
only showing top 10 rows



In [10]:
books_df = spark.read.options(sep=';', header=True, inferSchema=True).csv('/home/maryam/content/BooksRating-CSV/Books.csv')

In [18]:
books_df.show(2, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------
 ISBN              | 0195153448                                                   
 BookTitle         | Classical Mythology                                          
 BookAuthor        | Mark P. O. Morford                                           
 YearOfPublication | 2002                                                         
 Publisher         | Oxford University Press                                      
 ImageURLS         | http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg 
 ImageURLM         | http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg 
 ImageURLL         | http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg 
-RECORD 1-------------------------------------------------------------------------
 ISBN              | 0002005018                                                   
 BookTitle         | Clara Callan                                                 
 Boo

In [19]:
users_df = spark.read.options(sep=';', header=True, inferSchema=True).csv('/home/maryam/content/BooksRating-CSV/Users.csv')

In [21]:
users_df.show(10)

+------+-----------+--------------------+----+
|UserID|   USERNAME|            Location| Age|
+------+-----------+--------------------+----+
|     1|bzsufoRTLN2|  nyc, new york, usa|NULL|
|     2|fq7kfHg4VEI|stockton, califor...|  18|
|     3|W0Hbkd3xR8v|moscow, yukon ter...|NULL|
|     4|W51GahAx5Ap|porto, v.n.gaia, ...|  17|
|     5|VKN3PQ18GgN|farnborough, hant...|NULL|
|     6|h9BSgQZ5wOk|santa monica, cal...|  61|
|     7|7rdddpZpjWp| washington, dc, usa|NULL|
|     8|qiOJebWJS2i|timmins, ontario,...|NULL|
|     9|gkcxQJLS13A|germantown, tenne...|NULL|
|    10|BANPptNSbPy|albacete, wiscons...|  26|
+------+-----------+--------------------+----+
only showing top 10 rows



In [22]:
rating_df.show(10)

+------+----------+----+
|userid|      isbn|rate|
+------+----------+----+
|276725|034545104X|   0|
|276726|0155061224|   5|
|276727|0446520802|   0|
|276729|052165615X|   3|
|276729|0521795028|   6|
|276733|2080674722|   0|
|276736|3257224281|   8|
|276737|0600570967|   6|
|276744|038550120X|   7|
|276745| 342310538|  10|
+------+----------+----+
only showing top 10 rows



In [27]:
avg_rating_df = rating_df.groupBy("isbn").agg(avg("rate").alias("Avg"))

In [28]:
avg_rating_df.show(10)

[Stage 21:==============>                                           (1 + 3) / 4]

+----------+------------------+
|      isbn|               Avg|
+----------+------------------+
|2080674722|3.6666666666666665|
|3499134004|               0.0|
|3548603203|3.4166666666666665|
|880781112X| 4.333333333333333|
|0738205737|1.6666666666666667|
|0749317256|               0.0|
|0515131520|            1.6875|
|8471662531|               7.0|
|0441005470|               0.0|
|0739417096| 5.285714285714286|
+----------+------------------+
only showing top 10 rows



In [29]:
rating_new_df = rating_df.join(avg_rating_df, on="isbn", how="left")

In [30]:
rating_new_df.show(10)

[Stage 25:==============>                                           (1 + 3) / 4]

+----------+------+----+------------------+
|      isbn|userid|rate|               Avg|
+----------+------+----+------------------+
|0200001590|236283|   0|               0.0|
|034545104X|276725|   0| 2.933333333333333|
|0743436822| 79724|   0|2.1923076923076925|
|0743436830| 79724|   0|           1.96875|
|2080674722|276733|   0|3.6666666666666665|
| 342310538|276745|  10|               5.0|
|343011571X|160406|   0|               0.0|
|3442437407|160406|   0|          3.015625|
|3442545668|160406|   0|               0.0|
|0307000818|236283|   0|3.3333333333333335|
+----------+------+----+------------------+
only showing top 10 rows



In [35]:
rating_users_books_join= rating_new_df.join(users_df, 'userid').join(books_df, 'isbn').select('username', 'booktitle', 'rate', 'avg')

In [36]:
rating_users_books_join.show(10)

[Stage 62:==============>                                           (1 + 3) / 4]

+-----------+--------------------+----+---+
|   username|           booktitle|rate|avg|
+-----------+--------------------+----+---+
|6chdqlR3DC7|The Way Things Wo...|   8|8.0|
|cHwJip4Kj4k|                Liar|   9|9.0|
|6VUiynjA3tV|The Prime of Miss...|   0|0.0|
|cHwJip4Kj4k|    The Fighting Man|   9|9.0|
|a0EEWhgtsW8|  First Among Equals|   0|0.0|
|OMqCFWvTBPp|           Kidnapped|   0|0.0|
|cHwJip4Kj4k|Nothing Can Be Be...|   0|0.0|
|e8Hb8GyY8yN| Pearl and Sir Orfeo|   5|5.0|
|vRhKAqzPu1g|       Which Colour?|   0|0.0|
|1er18zMrBmP|Huck Scarry's Ste...|   0|0.0|
+-----------+--------------------+----+---+
only showing top 10 rows



In [ ]:
rating_users_books_join.write.csv("/home/maryam/content/rating_join_results.csv", header=True)

In [37]:
rating_new_df.createOrReplaceTempView("rating_new")
users_df.createOrReplaceTempView("users")
books_df.createOrReplaceTempView("books")

In [38]:
query = """
SELECT u.username, b.booktitle, r.rate, r.Avg
FROM rating_new r
JOIN users u ON r.userid = u.userid
JOIN books b ON r.isbn = b.isbn
"""

In [39]:
rating_users_books_join = spark.sql(query)

In [41]:
rating_users_books_join.show(10)

[Stage 78:==============>                                           (1 + 3) / 4]

+-----------+--------------------+----+---+
|   username|           booktitle|rate|Avg|
+-----------+--------------------+----+---+
|6chdqlR3DC7|The Way Things Wo...|   8|8.0|
|cHwJip4Kj4k|                Liar|   9|9.0|
|6VUiynjA3tV|The Prime of Miss...|   0|0.0|
|cHwJip4Kj4k|    The Fighting Man|   9|9.0|
|a0EEWhgtsW8|  First Among Equals|   0|0.0|
|OMqCFWvTBPp|           Kidnapped|   0|0.0|
|cHwJip4Kj4k|Nothing Can Be Be...|   0|0.0|
|e8Hb8GyY8yN| Pearl and Sir Orfeo|   5|5.0|
|vRhKAqzPu1g|       Which Colour?|   0|0.0|
|1er18zMrBmP|Huck Scarry's Ste...|   0|0.0|
+-----------+--------------------+----+---+
only showing top 10 rows



In [ ]:
rating_users_books_join.write.csv("/home/maryam/content/rating_users_books_join.csv", header=True)

In [ ]:
spark.stop()